In [23]:
import pandas as pd 
from sklearn import svm, datasets
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

iris = datasets.load_iris()
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['target'] = iris.target
df['target_1'] = df.target.apply(lambda x: iris.target_names[x])

In [24]:
df.drop('target',axis='columns',inplace=True)

ТЕПЕРЬ МОЖНО ПОДЕЛИТЬ ДАННЫЕ И ОБУЧИТЬ МОДЕЛЬ НЕСКОЛЬКИМИ СПОСОБАМИ:
1. TRAIN TEST SPLIT 
2. K FOLD CROSS
3. GridSearchCV

первый способ изи 
второй надо вспомнить

In [25]:
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'), iris.data, iris.target)
model = SVC()

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [28]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [26]:
cross_val_score(svm.SVC(kernel='poly',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])

так же существует еще один способ через циклы

In [30]:
import numpy as np

kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

GRID SEARCH SV

In [31]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00213647, 0.00085716, 0.0008956 , 0.00066504, 0.00086665,
        0.00066066]),
 'std_fit_time': array([1.34490657e-03, 2.12816938e-04, 8.93129617e-05, 3.15638106e-05,
        6.39728186e-05, 1.54137197e-05]),
 'mean_score_time': array([0.00128698, 0.00040979, 0.00051589, 0.00035653, 0.00046849,
        0.00036755]),
 'std_score_time': array([1.04484045e-03, 5.87306606e-05, 1.23391084e-04, 3.97980530e-06,
        3.23410211e-05, 1.82489761e-05]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'ker

In [34]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear','poly']
},cv=5, return_train_score=False)

clf.fit(iris.data,iris.target)
df = pd.DataFrame(clf.cv_results_)
df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001743,0.000265,0.000925,0.000230,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000916,0.000179,0.000611,0.000135,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.002220,0.001363,0.000467,0.000110,1,poly,"{'C': 1, 'kernel': 'poly'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [42]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel':['rbf','linear','poly']
},cv=5,return_train_score=False,n_iter=2)

In [46]:
rs.fit(iris.data,iris.target)
df = pd.DataFrame(rs.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001150,0.000409,0.000564,0.000196,linear,20,"{'kernel': 'linear', 'C': 20}",1.0,1.0,0.9,0.933333,1.0,0.966667,0.042164,1
1,0.006983,0.005664,0.000653,0.000373,poly,10,"{'kernel': 'poly', 'C': 10}",1.0,1.0,0.9,0.933333,1.0,0.966667,0.042164,1


In [47]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [51]:
model_params = {
    'svm':{
        'model':svm.SVC(gamma='auto'),
        'params':{
            'C':[1,10,20],
            'kernel':['rbf','linear']
        }
    },
    'random_forest':{
        'model':RandomForestClassifier(),
        'params':{
            'n_estimators':[1,5,10]
        }
    },
    'logistic_regression': {
        'model':LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,5,10]
        }
    }
}

In [52]:
scores = []

for model_name, mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params': clf.best_params_
    })

In [53]:
scores

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 1, 'kernel': 'rbf'}},
 {'model': 'random_forest',
  'best_score': 0.96,
  'best_params': {'n_estimators': 5}},
 {'model': 'logistic_regression',
  'best_score': 0.9666666666666668,
  'best_params': {'C': 5}}]

ДОМАШКА ДОМАШКА ДОМАШКА

In [54]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

In [77]:
models = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C': [5, 10, 20],
            'kernel': ['rbf', 'linear']
        }
    },
    'Random_Forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [100, 200]
        }
    },
    "LogisticRegression": {
        "model": LogisticRegression(max_iter=1000),  
        'params': {
            'C': [0.1, 1, 10], 
            'solver': ['lbfgs', 'liblinear']  
        }
    },
    'GaussianNB': {
        'model': GaussianNB(),
        'params': {}  
    },
    'MultinomialNB': {
        'model': MultinomialNB(),
        'params': {
            'alpha': [0.1, 1, 10] 
        }
    },
    'DecisionTreeClassifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [5, 10, 15],
            'max_leaf_nodes': [100, 200, 500]
        }
    }
}

In [78]:
scores = []

for model_name, mp in models.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)
    clf.fit(iris.data,iris.target)
    scores.append({
        'model':model_name,
        'best_score':clf.best_score_,
        'best_params': clf.best_params_
    })

In [79]:
scores

[{'model': 'svm',
  'best_score': 0.9800000000000001,
  'best_params': {'C': 5, 'kernel': 'rbf'}},
 {'model': 'Random_Forest',
  'best_score': 0.9600000000000002,
  'best_params': {'max_depth': 100, 'n_estimators': 100}},
 {'model': 'LogisticRegression',
  'best_score': 0.9733333333333334,
  'best_params': {'C': 1, 'solver': 'lbfgs'}},
 {'model': 'GaussianNB', 'best_score': 0.9533333333333334, 'best_params': {}},
 {'model': 'MultinomialNB',
  'best_score': 0.9666666666666668,
  'best_params': {'alpha': 10}},
 {'model': 'DecisionTreeClassifier',
  'best_score': 0.9666666666666668,
  'best_params': {'max_depth': 5, 'max_leaf_nodes': 200}}]